# Wino

In [1]:
from pandas import read_csv
from matplotlib.pyplot import subplots
from seaborn import kdeplot, histplot
from matplotlib.patches import Patch

X = read_csv('wino.csv')

## Histogramy

### Kwasowość

Na histogramach  widać, że kwasowość win czerwonych osiąga częściej większe wartosci, z kolei 
stężenie kwasu cytrynowego, który jak wiadomo wpływa na kwaskowatość, ma rozkład bardziej równomierny,
niż win białych, które zawierają go przeciętnie więcej.

Na podstawie wykresu można spekulować o **normlaności** rozkładu kwasowości, przynajmniej w przypadku
win **białych**.

### Cukier

Zawartość pozostałego cukru jest podobna, jednak na wykresach widać, że wina białe nierzadko mają go
więcej.

Wykres sugeruje, że wina czerwone mają zawartość cukru o rozkładzie **normalnym**.

### Wolne chlorki

Wolne chlorki w obu przypadkach mogą mieć rozkład przypominający rozkład **normalny**, wina czerwone 
mają ich jednak, przeciętnie, więcej.

### Dwutlenek siarki

Dwutlenek siarki ma widoczną asymetrię **lewostronną** i widać, że wina białe tych tlenków mają więcej,
także w przypadku całkowitego jego stężenia.

### Gęstość

Gęstości substancji zdają sie mieć rozkład **normalny**, szczególnie w przypadku win czerwonych, które
wydają się być także gęstsze.

### pH

Wartość pH na histogramie wskazuje, że wina czerwone są bardziej zasadowe, w obu przypadkach możemy
mieć do czynienia z rozkładem **normalnym**.

### Siarczany

Na siarczanach widać, że wina czerwone zawierają ich więcej, krzywa gęstości może przypominać krzywą
Gaussa rozkładu **normalnego**.

### Alkohol

Alkohol w obu przypadkach ma podobny rozkład, który nie przypomina rozkłądu normalnego.

### Jakość

Rozkład jakości zdaje się być podobny w przypadku obu rodzajów win.

In [2]:
def hist(col, ax, binwidth=None):

    ax.set_ylabel('%')
    if (binwidth is None):
        binwidth = 1 if col == 'jakość' else (X[col].max() - X[col].min())/20

    for is_red, group in X.groupby('czerwone'):

        histplot(data=group, x=col, ax=ax, binwidth = binwidth, 
            color = 'red' if is_red else 'lightgreen',
            stat="density", edgecolor="none", alpha=0.2
        )

        kdeplot(data=group, x=col, ax=ax, color='darkred' if is_red else 'green')

def hists():

    fig, axs = subplots(4,3, figsize=(20, 10), sharey=False)
    axs = axs.ravel()

    for i, col in enumerate(X.columns):
        if (col == 'czerwone'):
            continue
        hist(col, axs[i])

    fig.suptitle('Histogramy zmiennych dotyczących win czerwonych i białych', fontsize=16)
    fig.subplots_adjust(wspace=0.4, hspace=0.6)

    legend_elements = [Patch(facecolor='red', edgecolor='darkred',
                             label='Wina czerwone'),
                       Patch(facecolor='lightgreen', edgecolor='green',
                             label='Wina białe')]

    fig.legend(handles=legend_elements, loc='upper right')

hists()